### Add Oxford Museum of Natural History copyright acknowledgement

This script tags each image in a given directory with the following copyright acknowledgement:

**Acknowledgement: © Oxford University Museum of Natural History**

### Imports

In [9]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
import os
from os import path


### Create a FossilImage class

In [10]:
class FossilImage:
    def __init__(self, image_directory, image_filename):
        self.directory = image_directory
        self.filename = image_filename
        ## Set the font_size for the copyright banner
        self.font_size = 10
        ## Set the font to be used in the copyright banner
        self.banner_font = ImageFont.truetype("../../assets/OpenSans.ttf", self.font_size)
        ## Set the bottom position for the text within the banner image
        self.banner_bottom_line_offset = 16
        ## Set the spacing between text lines within the banner image
        self.line_space_offset = 12
        self.image = Image.open(image_directory + '/' + image_filename)
        self.width, self.height = self.image.size
        self.ensureMinimumSize()
        self.setBannerSizeAndText()
        self.banner_width = self.width
        self.tagged_image = None
        self.createTaggedImage()
        
    def ensureMinimumSize(self):
        '''
        Given an image that is smaller than 144 pixels wide, 
        resize it to at least 144 pixels wide while 
        maintaining its aspect ratio.  This is desirable in 
        order to use copyright text in the 10 point font 
        size which is the minimum font size which renders 
        cleanly.
        '''
        if self.width < 144:
            new_size_width = 144
            new_size_height = int(round((self.height * 144) / self.width))
            new_size = (new_size_width, new_size_height)
            self.image = self.image.resize(new_size, Image.BICUBIC)
            self.width, self.height = self.image.size

    def setBannerSizeAndText(self):
        '''
        Given an image, find its width and use that to decide how
        many lines will be needed to write out the copyright 
        acknowledgement text.  Then, return those lines of text in
        an array.
        '''
        if self.width < 200: # 3 lines
            self.text_lines = [
                ' Acknowledgement:',
                ' © Oxford University',
                ' Museum of Natural History'
            ]
            self.banner_height = 38

        elif self.width < 600:
            self.text_lines = [
                ' Acknowledgement: © Oxford University',
                ' Museum of Natural History'
            ]
            self.banner_height = 28
        else:
            self.text_lines = [
                ' Acknowledgement: © Oxford University Museum of Natural History'
            ]
            self.banner_height = 16

    def createTaggedImage(self):
        '''
        Perfom all the steps necessary to add the copyright
        acknowledgment to the image
        '''
        bottom_line_offset = 12
        # Create a new image for the banner
        banner_image = Image.new('1', (self.banner_width, self.banner_height))
        banner_drawing = ImageDraw.Draw(banner_image)
        
        # Add copyright to the banner image
        counter = len(self.text_lines) - 1
        for line in self.text_lines:
            this_line_offset = self.banner_bottom_line_offset + (counter * self.line_space_offset)
            banner_drawing.text(
                (0, self.banner_height - this_line_offset),
                line,
                font = self.banner_font,
                fill = 1
            )
            counter -=1
        
        # Create a new image "canvas" to concantenate the original image with the banner image
        tagged_image = Image.new('RGB', (self.width, self.height + self.banner_height))

        # Paste the original image onto the finished image "canvas"
        tagged_image.paste(self.image, (0, 0))

        # Paste the banner onto the finished image "canvas" underneath the original image
        tagged_image.paste(banner_image, (0, self.height))
        
        # Set the new image into the tagged_image property
        self.tagged_image = tagged_image
    
        

### Functions

In [11]:
def getListOfFiles(directory):
    '''
    Given a directory name, return a list of all the files
    within that directory or any subdirectories.
    '''
    files = []
    for (dirpath, dirnames, filenames) in os.walk(directory):
        files.extend(filenames)
        break
    # Remove any hidden files from the list
    for f in files:
        if f.startswith('.'):
            files.remove(f)
    # Return the list of files
    
    print(files)
    return files


### Add the copyright banner to each image file

In [12]:
# We will start a base directory that hold sub-directories organized by genus_species
# We will need to loop over this input_dir's subfolders and create each of them in the output_dir.
# Then, pass each matching pair of subdirectories through the process of tagging them with copyright text
# and saving the tagged image in the correct output subdirectory.

# Set the directory which contains the original images which
# need to be tagged with this copyright acknowledgement
input_images_dir_base  = '/Users/jp/Desktop/AAAAAA_Regorg/projects/Fossil_Image_Pipeline/image_data_cropped_and_converted'

# Set the directory into which the tagged images will be saved
# Usually this will be the same as the input directory so the 
# original images are overwritten with their tagged versions
# For testing though, you may wish to use a different directory
output_images_dir_base = '/Users/jp/Desktop/AAAAAA_Regorg/projects/Fossil_Image_Pipeline/image_data_cropped_and_converted_and_tagged'

# List the subdirectories of input_images_dir_base
list_input_subdirectories = [x[0] for x in os.walk(input_images_dir_base)]

for sd in list_input_subdirectories:
    # Establish this subdirectory's input and output paths
    input_images_dir = os.path.join(input_images_dir_base, sd)
    output_images_dir = os.path.join(output_images_dir_base, sd)
    
    # Make the output directory if it does not exist
    if not os.path.isdir(output_images_dir):
        os.mkdir(output_images_dir)
    
    # Get the list of fossil image files from the input_images_dir
    # specified in the Parameters section above.
    files = getListOfFiles(input_images_dir)

    # For each fossil image file, add the copyright banner, then save 
    # the finished image in the output_images_dir specified in the
    # Parameters section above.
    for file_name in files:

        fosImg = FossilImage(input_images_dir, file_name)

        # Show the finished image (optional)
        #fosImg.tagged_image.show()

        ## Save the finished image
        file_name_without_ext = file_name[:-4]
        fosImg.tagged_image.save(output_images_dir + '/' + file_name_without_ext + '.png')


['PAL-CL.00892b.png', 'PAL-CL.01742b.png']
['PAL-CL.01688.png']
['PAL-CL.01880d.png', 'PAL-CL.01880e.png', 'PAL-CL.01880b.png', 'PAL-CL.01880c.png', 'PAL-CL.01880a.png']
['PAL-CL.00109b.png', 'PAL-CL.00109c.png', 'PAL-CL.00109d.png']
['PAL-CL.01840a.png', 'PAL-CL.01840c.png', 'PAL-CL.01840b.png', 'PAL-CL.01840d.png', 'PAL-CL.01829c.png', 'PAL-CL.01829b.png', 'PAL-CL.01828a.png', 'PAL-CL.01829f.png', 'PAL-CL.01828b.png', 'PAL-CL.01828c.png', 'PAL-CL.01829g.png']
['PAL-J.068321.png']
['PAL-CL.00442d.png', 'PAL-CL.00442e.png', 'PAL-CL.00442b.png', 'PAL-CL.00442c.png', 'PAL-CL.00442a.png']
['PAL-J.16020.png']
['PAL-CL.01690w.png', 'PAL-CL.01900a.png', 'PAL-CL.01900b.png']
['PAL-CL.01792.png']
['PAL-CL.01709a.png', 'PAL-CL.01709b.png', 'PAL-CL.01701.png', 'PAL-CL.01262a.png', 'PAL-CL.01262b.png', 'PAL-CL.01262d.png']
['PAL-CL.00889.png', 'PAL-CL.00903.png', 'PAL-CL.00919c.png', 'PAL-CL.00919b.png', 'PAL-CL.00919a.png', 'PAL-CL.00901f.png', 'PAL-CL.00901d.png', 'PAL-CL.00901e.png', 'PAL-CL.0

['PAL-J.000857.png']
['PAL-J.068751.png', 'PAL-J.068745.png', 'PAL-J.068744.png', 'PAL-J.068750.png', 'PAL-J.068752.png', 'PAL-J.068747.png', 'PAL-J.068743.png', 'PAL-J.068754.png', 'PAL-J.068797.png', 'PAL-J.068836.png', 'PAL-J.068834.png', 'PAL-J.068798.png', 'PAL-J.068835.png', 'PAL-J.068749.png', 'PAL-J.068748.png']
['PAL-CL.03809a.png', 'PAL-CL.03809b.png']
['PAL-CL.01837b.png', 'PAL-CL.01837c.png', 'PAL-CL.01837a.png', 'PAL-CL.01837d.png', 'PAL-CL.01837e.png', 'PAL-CL.01837g.png', 'PAL-CL.01837f.png', 'PAL-CL.01837k.png', 'PAL-CL.01837j.png', 'PAL-CL.01837h.png', 'PAL-CL.01837i.png', 'PAL-CL.01834a.png', 'PAL-CL.01837m.png', 'PAL-CL.01837l.png', 'PAL-CL.01834b.png', 'PAL-CL.01837n.png', 'PAL-CL.01837o.png']
['PAL-J.025780.png', 'PAL-J.029717.png']
['PAL-CL.04222.png']
['PAL-J.029714.png']
['PAL-CL.04223.png']
['PAL-CL.01817a.png', 'PAL-CL.01817b.png', 'PAL-CL.01813.png']
['PAL-J.000941.png']
['PAL-CL.01892a.png', 'PAL-CL.01892b.png']
['PAL-J.20309.png']
['PAL-CL.04216b.png', 'PAL

['PAL-CL.04226b.png', 'PAL-CL.04226a.png']
['PAL-CL.01722.png']
['PAL-J.04328.png']
['PAL-CL.01807b.png', 'PAL-CL.01807a.png']
['PAL-J.000952.png']
['PAL-J.02998.png']
['PAL-J.02983.png']
['PAL-CL.03842a.png', 'PAL-CL.03842b.png', 'PAL-CL.03842c.png']
['PAL-CL.01875a.png', 'PAL-CL.01875b.png']
['PAL-CL.01759a.png', 'PAL-CL.01759b.png']
['PAL-CL.01284e.png', 'PAL-CL.01284f.png', 'PAL-CL.01284b.png', 'PAL-CL.01284c.png', 'PAL-CL.01284f.png.png', 'PAL-CL.01284f.png.sb-f4975b0b-5quThW']
['PAL-CL.00427b.png', 'PAL-CL.00427c.png', 'PAL-CL.00427a.png']
['PAL-J.068319.png']
['PAL-CL.01854a.png', 'PAL-CL.01854b.png', 'PAL-CL.01854c.png']
['PAL-J.20843.png']
['PAL-CL.01269e.png', 'PAL-CL.01269d.png', 'PAL-CL.01269f.png', 'PAL-CL.01269b.png']
['PAL-CL.01771.png', 'PAL-CL.01763a.png', 'PAL-CL.01763b.png']
['PAL-CL.01782b.png', 'PAL-CL.01782c.png', 'PAL-CL.01782a.png']
['PAL-CL.01876b.png', 'PAL-CL.01876c.png', 'PAL-CL.01876a.png', 'PAL-CL.01876d.png', 'PAL-CL.01876e.png', 'PAL-CL.01879.png', 'PAL-

['PAL-CL.01849c.png', 'PAL-CL.01849b.png', 'PAL-CL.01849a.png']
['PAL-CL.01705c.png', 'PAL-CL.01705b.png', 'PAL-CL.01705a.png', 'PAL-CL.01704.png']
['PAL-J.02974.png']
['PAL-CL.04225a.png', 'PAL-CL.04225c.png', 'PAL-CL.04225b.png']
['PAL-CL.01689g.png']
['PAL-CL.01765.png']
['PAL-J.001345.png', 'PAL-J.001303.png']
['PAL-CL.01728a.png', 'PAL-CL.01728c.png', 'PAL-CL.01728b.png', 'PAL-CL.01729b.png', 'PAL-CL.01729c.png', 'PAL-CL.01729a.png', 'PAL-CL.01731.png']
['PAL-CL.01738a.png']
['PAL-CL.00926b.png', 'PAL-CL.00926c.png', 'PAL-CL.00926a.png', 'PAL-CL.00926d.png']
['PAL-J.04570.png']
['PAL-CL.00108e.png', 'PAL-CL.00108d.png', 'PAL-CL.00108f.png', 'PAL-CL.00108g.png', 'PAL-CL.00108c.png', 'PAL-CL.00108b.png', 'PAL-CL.00108a.png', 'PAL-CL.00108l.png', 'PAL-CL.00108m.png', 'PAL-CL.00108o.png', 'PAL-CL.00108j.png', 'PAL-CL.00108k.png', 'PAL-CL.00108i.png', 'PAL-CL.00108h.png']
['PAL-CL.01816.png']
['PAL-J.04331.png']
['PAL-CL.04231.png']
['PAL-J.04006.png']
['PAL-J.04072.png']
['PAL-CL.0041